<a href="https://colab.research.google.com/github/RohanGforwork/ML_text_summarization/blob/main/team_1_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/datasets_foss/twcs.csv')
df

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
...,...,...,...,...,...,...,...
2811769,2987947,sprintcare,False,Wed Nov 22 08:43:51 +0000 2017,"@823869 Hey, we'd be happy to look into this f...",NaN,2987948.0
2811770,2987948,823869,True,Wed Nov 22 08:35:16 +0000 2017,@115714 wtf!? I’ve been having really shitty s...,2987947,NaN
2811771,2812240,121673,True,Thu Nov 23 04:13:07 +0000 2017,@143549 @sprintcare You have to go to https://...,NaN,2812239.0
2811772,2987949,AldiUK,False,Wed Nov 22 08:31:24 +0000 2017,"@823870 Sounds delicious, Sarah! 😋 https://t.c...",NaN,2987950.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811774 entries, 0 to 2811773
Data columns (total 7 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tweet_id                 int64  
 1   author_id                object 
 2   inbound                  bool   
 3   created_at               object 
 4   text                     object 
 5   response_tweet_id        object 
 6   in_response_to_tweet_id  float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 131.4+ MB


In [5]:
df.isnull().sum()

,0
tweet_id,0
author_id,0
inbound,0
created_at,0
text,0
response_tweet_id,1040629
in_response_to_tweet_id,794335


In [6]:
df = df[[ 'text']]

In [7]:
print(df['text'].dtype)
print(df.info())

object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811774 entries, 0 to 2811773
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   text    object
dtypes: object(1)
memory usage: 21.5+ MB
None


In [8]:
print(df.head())

                                                text
0  @115712 I understand. I would like to assist y...
1      @sprintcare and how do you propose we do that
2  @sprintcare I have sent several private messag...
3  @115712 Please send us a Private Message so th...
4                                 @sprintcare I did.


In [9]:
'''pip install datasets'''

'pip install datasets'

In [1]:
import pandas as pd
import re
import string
import spacy
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/datasets_foss/twcs.csv')

nlp = spacy.load("en_core_web_sm")

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation

    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop])  # Lemmatize & remove stopwords

# Apply cleaning function
df["cleaned_text"] = df["text"].apply(clean_text)

# Train-Test Split
train_texts, test_texts = train_test_split(df["cleaned_text"], test_size=0.2, random_state=42)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/datasets_foss/twcs.csv'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/long-t5-local-base")

# Tokenize training and testing data
train_encodings = tokenizer(list(train_texts), padding=True, truncation=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer(list(test_texts), padding=True, truncation=True, max_length=512, return_tensors="pt")


In [ ]:
# long t5 conditional generation
from transformers import LongT5ForConditionalGeneration, Trainer, TrainingArguments
import torch

model = LongT5ForConditionalGeneration.from_pretrained("google/long-t5-local-base")

# Convert data to PyTorch tensors
train_labels = tokenizer(list(train_texts), padding=True, truncation=True, max_length=150, return_tensors="pt").input_ids
test_labels = tokenizer(list(test_texts), padding=True, truncation=True, max_length=150, return_tensors="pt").input_ids

train_data = [{"input_ids": train_encodings.input_ids[i], "attention_mask": train_encodings.attention_mask[i], "labels": train_labels[i]} for i in range(len(train_texts))]
test_data = [{"input_ids": test_encodings.input_ids[i], "attention_mask": test_encodings.attention_mask[i], "labels": test_labels[i]} for i in range(len(test_texts))]

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)

# Start Training
trainer.train()


In [ ]:
model.save_pretrained("./fine_tuned_longt5")
tokenizer.save_pretrained("./fine_tuned_longt5")


In [ ]:
def summarize_tweet(tweet, model, tokenizer):
    input_ids = tokenizer.encode("summarize: " + tweet, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(input_ids, max_length=150, num_beams=3, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

test_tweet = "Your sample social media text here..."
print("Summary:", summarize_tweet(test_tweet, model, tokenizer))


In [ ]:
import spacy
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load Summarization Models
pegasus_model_name = "google/pegasus-xsum"
longt5_model_name = "google/long-t5-tglobal-base"

pegasus_tokenizer = AutoTokenizer.from_pretrained(pegasus_model_name)
pegasus_model = AutoModelForSeq2SeqLM.from_pretrained(pegasus_model_name)

longt5_tokenizer = AutoTokenizer.from_pretrained(longt5_model_name)
longt5_model = AutoModelForSeq2SeqLM.from_pretrained(longt5_model_name)

# Function to clean text using spaCy
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation

    doc = nlp(text)  # Process text with spaCy
    cleaned_text = " ".join([token.lemma_ for token in doc if not token.is_stop])  # Lemmatize & remove stopwords
    return cleaned_text

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/datasets_foss/twcs.csv')

# Apply text cleaning
df['cleaned_text'] = df['text'].apply(clean_text)

# Convert text to TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Limit vocabulary size for efficiency
tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_text'])

# Function to summarize text dynamically
def summarize_text(text):
    text_length = len(text)

    # Select model dynamically
    if text_length <= 300:
        tokenizer, model = pegasus_tokenizer, pegasus_model
        max_len = 128  # Shorter summary for tweets
    else:
        tokenizer, model = longt5_tokenizer, longt5_model
        max_len = 200  # Longer summary for paragraphs

    # Tokenize input
    input_ids = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

    # Generate summary
    summary_ids = model.generate(input_ids, max_length=max_len, num_beams=5, early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Apply summarization function
df['summary'] = df['cleaned_text'].apply(summarize_text)

# Display results
print(df[['text', 'summary']].head())


In [ ]:
df['text'][142592]

In [ ]:
df

In [ ]:
'''!pip install transformers[torch]
!pip install accelerate -U'''

In [ ]:
import os

In [ ]:
from transformers import LongT5ForConditionalGeneration, AutoTokenizer

model = LongT5ForConditionalGeneration.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")
tokenizer = AutoTokenizer.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")

def summarize_tweet(tweet, model=model, tokenizer=tokenizer):
    # Preprocess and tokenize the tweet
    input_ids = tokenizer.encode("summarize: " + tweet, return_tensors="pt", max_length=1000, truncation=True)

    # Generate summary using the LongT5 model
    summary_ids = model.generate(input_ids, max_length=512, num_beams=1, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

tweet = input("Enter a tweet to summarize: ")
summary = summarize_tweet(tweet)

print("Original tweet:", tweet)
print("Summary:", summary)